## Jobs Handling Example
Show the functions relates to the stopping of Jobs.

In [ ]:
import asyncio
import time

import corex
from corex import TrainInput, PredictInput
from corex.jobs import DataUpload, Experiment
from corex.corex_vars import CoreXStatus

In [ ]:
# set the username, password, host
corex.enable_default_logger()
context = corex.Context.create(
        username='{usr}', password='{pwd}', host='{corexserver}')
client = corex.CoreXAPI()

In [ ]:
train_file_path = os.path.join('{train_file_path}')
test_file_path = os.path.join('{test_file_path}')
train_file = open(train_file_path , 'r')
test_file = open(test_file_path , 'r')

## Job.stop() in diff status

Job failed if it is stopped in running status.

In [ ]:
train_data = client.upload(file=train_file, name='stop_when_running')

async def stop_when_running():
    while train_data.status == 'pending':
        await asyncio.sleep(3)
    train_data.stop()
    return

await stop_when_running()

Job failed if it is stopped in pending status.

In [ ]:
train_data = DataUpload(name='stop_when_pending')
train_data.stop()
print(train_data.status)

Job remain failed if it is stopped in pending status.

In [ ]:
train_data = DataUpload(name='stop_when_failed')
train_data.stop()
print(train_data.status)
train_data.stop()

## Test dependency
Stop DataUpload, then the Xxperiemnt and predict depends on the data will fail. Test data will keep upload till it's done.

In [ ]:
train_file = open(train_file_path , 'r')
test_file = open(test_file_path, 'r')

train_data = client.upload(file=train_file, name='train_data')
test_data = client.upload(file=test_file, name='test_data')

train_input = TrainInput(data=train_data, target='Survived', algos=['XGBoost'], max_model=2, tolerance=0.9)
exp = client.train(train_input=train_input, select_model_by='auc', name='exp(data_failed)')

predict_input = PredictInput(data=test_data, experiment=exp)

# predict result
pred_res = client.predict(predict_input=predict_input, name='pred(exp_failed)')
    
async def stop():
    while train_data.status == CoreXStatus.PENDING:
        await asyncio.sleep(3)
    
    train_data.stop()
    return

await stop()

## Test stop jobs list
Stop list of jobs, the jobs exclude in list will keep continue till done.

In [ ]:
train_file = open(train_file_path , 'r')

train_data = client.upload(file=train_file, name='train_data')
train_input = TrainInput(data=train_data, target='Survived', algos=['XGBoost'], max_model=2, tolerance=0.9)

exps = []
for i in range(5):
    exp = client.train(train_input=train_input, select_model_by='auc', name='exp_%d' % i)
    exps.append(exp)

time.sleep(4)
context.stop_jobs([exps[0], exps[2], exps[4]])


Get status of all the Jobs.

In [ ]:
context.get_jobs_status()

## Test stop all jobs
Stop all jobs while data is done, all the experiment should be stopped

In [ ]:
train_file = open(train_file_path , 'r')

train_data = client.upload(file=train_file, name='train_data')

train_input = TrainInput(data=train_data, target='Survived', algos=['XGBoost'], max_model=2, tolerance=0.9)

exps = []
for i in range(5):
    exp = client.train(train_input=train_input, select_model_by='auc', name='exp_%d' % i)
    exps.append(exp)

async def stop():
    while train_data.status not in CoreXStatus.DONE_STATUS:
        await asyncio.sleep(3)
    
    context.stop_all_jobs()
    return

await stop()


In [ ]:
context.get_jobs_status()